# UDACITY SageMaker Essentials: Processing Job Exercise

In prior exercises, we've been running and rerunning the same preprocessing job over and over again. For cleanly formatted data, it's possible to do some preprocessing utilizing batch transform. However, if slightly more sophisticated processing is needed, we would want to do so through a processing job. Finally, after beating around the bush for a few exercises, we're finally going offload the preprocessing step of our data!

In [1]:
import sklearn
import boto3
import json

## Preprocessing (for the final time!)

The cell below should be very familiar to you by now. This cell will write the preprocessing code to a file called "HelloBlazePreprocess.py". This code will be utilized by AWS via a SciKitLearn processing interface to process our data. 

In [2]:
%%writefile HelloBlazePreprocess.py

import json
import zipfile

# Function below unzips the archive to the local directory. 

def unzip_data(input_data_path):
    with zipfile.ZipFile(input_data_path, 'r') as input_data_zip:
        input_data_zip.extractall('.')
        return input_data_zip.namelist()[0]

# Input data is a file with a single JSON object per line with the following format: 
# {
#  "reviewerID": <string>,
#  "asin": <string>,
#  "reviewerName" <string>,
#  "helpful": [
#    <int>, (indicating number of "helpful votes")
#    <int>  (indicating total number of votes)
#  ],
#  "reviewText": "<string>",
#  "overall": <int>,
#  "summary": "<string>",
#  "unixReviewTime": <int>,
#  "reviewTime": "<string>"
# }
# 
# We are specifically interested in the fields "helpful" and "reviewText"
#

def label_data(input_data):
    labeled_data = []
    HELPFUL_LABEL = "__label__1"
    UNHELPFUL_LABEL = "__label__2"
     
    for l in open(input_data, 'r'):
        l_object = json.loads(l)
        helpful_votes = float(l_object['helpful'][0])
        total_votes = l_object['helpful'][1]
        reviewText = l_object['reviewText']
        if total_votes != 0:
            if helpful_votes / total_votes > .5:
                labeled_data.append(" ".join([HELPFUL_LABEL, reviewText]))
            elif helpful_votes / total_votes < .5:
                labeled_data.append(" ".join([UNHELPFUL_LABEL, reviewText]))
          
    return labeled_data


# Labeled data is a list of sentences, starting with the label defined in label_data. 

def split_sentences(labeled_data):
    new_split_sentences = []
    for d in labeled_data:
        label = d.split()[0]        
        sentences = " ".join(d.split()[1:]).split(".") # Initially split to separate label, then separate sentences
        for s in sentences:
            if s: # Make sure sentences isn't empty. Common w/ "..."
                new_split_sentences.append(" ".join([label, s]))
    return new_split_sentences

def write_data(data, train_path, test_path, proportion):
    border_index = int(proportion * len(data))
    train_f = open(train_path, 'w')
    test_f = open(test_path, 'w')
    index = 0
    for d in data:
        if index < border_index:
            train_f.write(d + '\n')
        else:
            test_f.write(d + '\n')
        index += 1

if __name__ == "__main__":
    unzipped_path = unzip_data('/opt/ml/processing/input/Toys_and_Games_5.json.zip')
    labeled_data = label_data(unzipped_path)
    new_split_sentence_data = split_sentences(labeled_data)
    write_data(new_split_sentence_data, '/opt/ml/processing/output/train/hello_blaze_train_scikit', '/opt/ml/processing/output/test/hello_blaze_test_scikit', .9)

Writing HelloBlazePreprocess.py


## Exercise: Upload unprocessed data to s3

No more local preprocessing! Upload the **raw zipped** Toys_and_Games dataset to s3. 

In [7]:
import os 
import boto3
from botocore.exceptions import ClientError

# Todo
s3_bucket = "udacity-xin-mlworkflow"
s3_prefix = "lesson2/exercise4"
file_name = "Toys_and_Games_5.json.zip"

def upload_file_to_s3(file_name):
    object_name = os.path.join(s3_prefix, file_name)
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, s3_bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False

upload_file_to_s3(file_name)
source_path = "s3://" + "/".join([s3_bucket, s3_prefix, file_name])
print(source_path)

udacity-xin-mlworkflow/lesson2/exercise4/Toys_and_Games_5.json.zip


## Exercise: Launch a processing job through the SciKitLearn interface. 

We'll be utilizing the SKLearnProcessor object from SageMaker to launch a processing job. Here is some information you'll need to complete the exercise: 

* You will need to use the SKLearnProcessor object. 
* You will need 1 instance of ml.m5.large. You can specify this programatically. 
* You will need an execution role.  

* You will need to call the "run" method on the SKLearnProcessor object.
> * You will need to specify the preprocessing code
> * the S3 path of the unprocessed data
> * a 'local' directory path for the input to be downloaded into
> * 'local' directory paths for where you expect the output to be.

you will need to make use of the ProcessingInput and ProcessingOutput features. Review the preprocessing code for where the output is going to go, and where it expects the input to come from.  
* It is highly recommended that you consult the documentation to help you implement this. https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html

Remember that, conceptually, you are creating a server that our code will be run from. This server will download data, execute code that we specify, and upload data to s3. 

If done successfully, you should see a processing job launch in the SageMaker console. To see it, go to the "processing" drop-down menu on the left-hand side and select "processing jobs." Wait until the job is finished. 

In [12]:
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Get role

role = get_execution_role()

# Create an SKLearnProcessor. Set framework_version='0.20.0'.
source_path = "s3://" + "/".join([s3_bucket, s3_prefix, file_name])
sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.m5.large',
                                     instance_count=1)

# Start a run job. You will pass in as parameters the local location of the processing code, 
# a processing input object, two processing output objects. The paths that you pass in here are directories, 
# not the files themselves. Check the preprocessing code for a hint about what these directories should be. 

sklearn_processor.run(code= 'HelloBlazePreprocess.py', # preprocessing code
                      inputs=[ProcessingInput(
                          source = source_path, # the S3 path of the unprocessed data
                          destination='/opt/ml/processing/input', # a 'local' directory path for the input to be downloaded into
                      )],
                      outputs=[ProcessingOutput(source='/opt/ml/processing/output/train'),# a 'local' directory path for where you expect the output for train data to be
                               ProcessingOutput(source='/opt/ml/processing/output/test')]) # a 'local' directory path for where you expect the output for test data to be 

[03/22/25 20:13:14] INFO     Defaulting to only available Python version: py3                     ]8;id=100790;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=20743;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#610\610]8;;\

[03/22/25 20:13:15] INFO     Creating processing-job with name                                      ]8;id=68416;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=32299;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1575\1575]8;;\
                             sagemaker-scikit-learn-2025-03-22-20-13-14-895                                        

...............
.

## Exercise: Sanity Check & Reflection. 

If all goes well, processed data should have been uploaded to S3. If you're having trouble locating the uri, check the `jobs` attribute of the SKLearnProcessor object. 

In [13]:
sklearn_processor.jobs[-1].describe()

{'ProcessingInputs': [{'InputName': 'input-1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://udacity-xin-mlworkflow/lesson2/exercise4/Toys_and_Games_5.json.zip',
    'LocalPath': '/opt/ml/processing/input',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'code',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-510096416228/sagemaker-scikit-learn-2025-03-22-20-13-14-895/input/code/HelloBlazePreprocess.py',
    'LocalPath': '/opt/ml/processing/input/code',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'output-1',
    'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-510096416228/sagemaker-scikit-learn-2025-03-22-20-13-14-895/output/output-1',
     'LocalPath': '/opt/ml/processing/output/train',
     'S

Download these datasets and compare them to the datasets that we locally processed. The exact sentences in the training & the test sets may vary depending on your implementation, but the same number of sentences should be present in each job, and there should be one label and one sentence per line.  


Once you've confirmed that the data was accurately processed, take a step back and reflect on what you've done. You've created the individual components necessary to process data, train data, and perform inference on both individual instances of data and large datasets. What are we missing if we wanted to provide a live, continuous service? Keep this question in mind as we move on to designing workflows. 